In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Importing custom functions defined in separate Python files
import genw_sparse
import run_target
import run_train
import run_test

# Define the size of the network and the connectivity parameters
N = 200  # Number of neurons
p = 0.3  # Probability of having a connection between two neurons
g = 1.0  # Scaling factor for weights

# Neuron dynamics parameter
b = 1/2  # Parameter related to the neuron dynamics, possibly a bias term

# Training parameters
train_duration = 100.0  # Duration of the training phase
nloop = 10  # Number of training loops (can be adjusted)
lambda_ = 0.5  # Regularization parameter (can be adjusted)
learn_every = 1.0  # Frequency of learning updates

# Stimulation parameters
stim_on = 100.0  # Time when the stimulus turns on
stim_off = 110.0  # Time when the stimulus turns off
train_time = stim_off + train_duration  # Total training time
test_time = train_time + 100.0  # Total time for testing phase

# Time step for simulation
dt = 0.1  # Time step in the simulation

# Choose the type of target for the training
target_type = "ratemodel"  # Target type (e.g., "periodic", "ou", "ratemodel")

# Generate the initial weight matrix (connectivity matrix)
M = genw_sparse.genw_sparse(N, 0, 1, p) / np.sqrt(N * p)

# Generate a random stimulus
stim = 2 * (2 * np.random.rand(N) - 1)

# Set up parameters for the network and generate target patterns
net_param = [N, p, g, b, dt, test_time]
utarg = run_target.run_target(M, net_param, target_type)

# Training phase
# Initialize initial conditions for the network
x0 = 2 * np.pi * np.random.rand(N)
r0 = np.zeros(N)

# Set up training parameters
net_param = [x0, r0, b]
train_param = [stim, lambda_, learn_every, nloop, target_type]
time_param = [stim_on, stim_off, train_time]

# Run the training process
M = run_train.run_train(M, net_param, train_param, time_param, dt, utarg)

# Testing phase
# Reinitialize initial conditions
x0 = 2 * np.pi * np.random.rand(N)
r0 = np.zeros(N)
z0 = np.zeros(N)

# Set up parameters for testing
net_param = [x0, r0, b, stim]
time_param = [stim_on, stim_off, test_time]

# Run the testing process
utest = run_test.run_test(M, net_param, time_param, dt)

# Plotting the results
# Create a time vector for plotting
tvec = np.arange(dt, test_time + dt, dt)

# Set up the plot
plt.figure(figsize=(10, 5))
nrow, ncol = 2, 2  # Number of rows and columns in the subplot

# Loop over each subplot
for i in range(nrow * ncol):
    plt.subplot(2, 2, i + 1)
    plt.axvspan(10 * stim_on, 10 * stim_off, alpha=0.5, color="dodgerblue")
    plt.axvspan(10 * stim_off, 10 * train_time, alpha=0.5, color="gray")

    # Plot target and actual patterns during the stimulation period
    ton = int(stim_off / dt)
    toff = int(train_time / dt)
    plt.plot(tvec[ton:toff] * 10, utarg[ton:toff, i], linewidth=3, color="black", label="target")
    plt.plot(tvec * 10, utest[:, i], linewidth=2, color="red", alpha=1, label="actual")

    # Set plot limits and labels
    plt.xlim([500, 2500])
    plt.xlabel("time (ms)", fontsize=15)
    plt.ylabel("synaptic current", fontsize=15)

    # Add legend to the last subplot
    if i == 3:
        plt.legend(loc=1, frameon=False, fontsize=15)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()
